# Table of Contents
**1. Away Score Predictor**
- Training model on historical data
- Model evaluation

**2. Home Score Predictor**
- Training model on historical data
- Model evaluation

**3. Week 12 Predictions for 2023 Season**
- Predicting the outcome of NFL games during Week 12 of 2023
- Comparing predictions to outcomes of week 12 of 2023

In [1]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA
from sklearn.metrics import r2_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

/Users/justintunley/anaconda3/envs/bigdata_ml/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-11-29 14:55:40.092776: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Training and Evaluating Away Score Predictor

In [3]:
hist = pd.read_csv('/Users/justintunley/Documents/BrainStation/Capstone/FINAL_NET_TRAINING.csv', index_col='summary')

In [4]:
hist.head()

,PPG_Team1,Tot_TDs_PG_Team1,1st_Downs_PG_Team1,Rush_1st_Downs_PG_Team1,Pass_1st_Downs_PG_Team1,OFF_1st_by_pen_PG_Team1,3rd_Conv_Rate_Team1,4th_Conv_Rate_Team1,Pass_Comp_Rate_Team1,Pass_Yds_PG_Team1,...,Pass_Att_PG_Team2,DEF_Pass_Att_PG_Team2,DEF_Sacks_PG_Team2,DEF_Sack_Yds_PG_Team2,DEF_FG_Att_PG_Team2,DEF_FG_Good_PG_Team2,season,away_score,home_score,total_points
summary,,,,,,,,,,,,,,,,,,,,,
ind_ne2004,32.6,4.1250,23.6875,5.8750,14.8750,2.9375,42.68,57.14,0.669829,288.9375,...,30.3125,33.6250,2.8125,19.4375,1.125,0.9375,2004,24.0,27.0,51.0
ten_mia2004,21.5,2.5625,19.2500,5.3125,12.5000,1.4375,34.10,44.44,0.604414,226.0000,...,36.6250,27.1250,2.2500,13.9375,1.750,1.2500,2004,17.0,7.0,24.0
jax_buf2004,17.4,1.6250,17.4375,5.5000,10.0625,1.8750,36.87,55.56,0.594542,197.4375,...,28.8125,30.3750,2.8125,19.9375,2.000,1.6875,2004,13.0,10.0,23.0
det_chi2004,18.5,2.0000,16.4375,5.7500,8.8750,1.8125,31.43,37.50,0.564356,182.2500,...,29.4375,32.1875,2.1875,10.8125,2.250,1.6250,2004,20.0,16.0,36.0
bal_cle2004,19.8,2.0625,16.2500,6.4375,8.4375,1.3750,35.07,26.67,0.554839,144.5000,...,27.4375,28.7500,2.0000,11.8750,1.750,1.5000,2004,3.0,20.0,23.0


In [163]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = hist.drop(columns = ['total_points', 'away_score', 'home_score', 'Year_Team1', 'Year_Team2'])
y = hist['away_score']
indices = hist.index

X_train, X_test, y_train, y_test, index_train, index_test = train_test_split(X, y, indices, test_size = 0.33)

scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [127]:
X.shape
# 3870 x 153 is correct

(3870, 153)

In [128]:
type(X_test)
# should be numpy array

numpy.ndarray

In [164]:
model = keras.Sequential()
# model layout for away team

model.add(layers.Dense(120, activation="sigmoid"))
model.add(layers.Dense(80, activation="sigmoid"))
model.add(layers.Dense(60, activation="relu"))
model.add(layers.Dense(40, activation="sigmoid"))
model.add(layers.Dense(20, activation="sigmoid"))
model.add(layers.Dense(40, activation="sigmoid"))
model.add(layers.Dense(60, activation="relu"))
model.add(layers.Dense(80, activation="sigmoid"))
model.add(layers.Dense(120, activation="sigmoid"))

# when should I use which activate? How dense?
# try a gridsearch?

model.add(layers.Dense(1, activation=None))

model.compile(
    # Optimizer
    optimizer=keras.optimizers.Adam(),
    # Loss function to minimize
    loss=keras.losses.MeanSquaredError()
)

# Beginning of Experiment

In [12]:
# model = keras.Sequential()
# # is keras the right one?

# model.add(layers.Dense(120, activation="relu"))
# model.add(layers.Dense(80, activation="relu"))
# model.add(layers.Dense(60, activation="relu"))
# model.add(layers.Dense(40, activation="relu"))
# model.add(layers.Dense(20, activation="relu"))
# model.add(layers.Dense(40, activation="relu"))
# model.add(layers.Dense(60, activation="relu"))
# model.add(layers.Dense(80, activation="relu"))
# model.add(layers.Dense(120, activation="relu"))

# # when should I use which activate? How dense?
# # try a gridsearch?

# model.add(layers.Dense(1, activation=None))

# model.compile(
#     # Optimizer
#     optimizer=keras.optimizers.Adam(),
#     # Loss function to minimize
#     loss=keras.losses.MeanSquaredError()
# )

In [133]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [153]:

# batch_size = [1,32,64,128]
epochs=[40,60,80,100,200]
activation = ['softmax', 'softplus', 'softsign', 'sigmoid', 'tanh', 'relu', 'linear', 'hard_sigmoid']

param_grid = dict(epochs=epochs, model__activation=activation)  # batch_size=batch_size
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, verbose=1)

In [154]:
grid_result = grid.fit(X_train, y_train)

TypeError: If no scoring is specified, the estimator passed should have a 'score' method. The estimator <keras.src.engine.sequential.Sequential object at 0x7fcbcbe46be0> does not.

In [144]:
grid.best_params_


AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

# End of Experiment

In [165]:
AwayPredictor = model.fit(X_train, y_train, epochs=50, verbose=1)

Epoch 1/50
81/81 [==============================] - 1s 2ms/step - loss: 357.8924
Epoch 2/50
81/81 [==============================] - 0s 1ms/step - loss: 180.4331
Epoch 3/50
81/81 [==============================] - 0s 1ms/step - loss: 127.8828
Epoch 4/50
81/81 [==============================] - 0s 1ms/step - loss: 107.2845
Epoch 5/50
81/81 [==============================] - 0s 1ms/step - loss: 101.2114
Epoch 6/50
81/81 [==============================] - 0s 1ms/step - loss: 99.9305
Epoch 7/50
81/81 [==============================] - 0s 1ms/step - loss: 99.7366
Epoch 8/50
81/81 [==============================] - 0s 1ms/step - loss: 99.7158
Epoch 9/50
81/81 [==============================] - 0s 1ms/step - loss: 99.7122
Epoch 10/50
81/81 [==============================] - 0s 1ms/step - loss: 99.7326
Epoch 11/50
81/81 [==============================] - 0s 1ms/step - loss: 99.7204
Epoch 12/50
81/81 [==============================] - 0s 1ms/step - loss: 99.7169
Epoch 13/50
81/81 [=============

In [166]:
train_loss = AwayPredictor.history["loss"][-1]
result = model.evaluate(X_test, y_test, verbose=0)

print(f"Train Loss: {train_loss:.4f}")
print(f"Test Loss: {result:.4f}")

a_predictions = model.predict(X_test)

Train Loss: 75.4086
Test Loss: 87.5195
40/40 [==============================] - 0s 1ms/step


In [167]:
from sklearn.metrics import mean_squared_error

away_mse = mean_squared_error(y_test,a_predictions)
away_mse

87.51950954269194

In [168]:
away_rmse = np.sqrt(away_mse)
away_rmse

9.35518623773423

In [99]:
a_predictions.shape

(1278, 1)

In [100]:
index_test.shape

(1278,)

In [169]:
AwayPredictions = pd.DataFrame(a_predictions, index=index_test)

In [170]:
AwayPredictions = AwayPredictions.rename(columns={0: 'total_points'})

In [171]:
AwayPredictions = AwayPredictions.rename(columns={'total_points': 'away_points_prediction'})

In [172]:
AwayPredictions.head()

,away_points_prediction
summary,
buf_den2014,15.913320
sea_sf2009,15.907855
nyj_buf2007,15.914616
cle_ind2011,19.318775
car_tb2017,26.043144


In [173]:
def get_away_score(away_team, home_team, season):
    temp_index = away_team + '_' + home_team + season
    result = AwayPredictions.loc[temp_index,'away_points_prediction']
    return result

In [175]:
score = get_away_score('nyj', 'buf', '2007')
score
# the Jets won in OT, 16-13. Thats very very close 

15.914616

In [176]:
AwayPredictions.describe()

,away_points_prediction
count,1278.000000
mean,21.760979
std,4.862992
min,15.906731
25%,15.911899
50%,25.808719
75%,26.067198
max,26.078146


In [32]:
# lets make our home score predictor before comparing with week 12

## Training and Evaluating Home Score Predictor

In [236]:
X2 = hist.drop(columns = ['total_points', 'away_score', 'home_score', 'Year_Team1', 'Year_Team2'])
y2 = hist['home_score']
indices2 = hist.index

X2_train, X2_test, y2_train, y2_test, index2_train, index2_test = train_test_split(X2, y2, indices, test_size = 0.33)

In [237]:
scaler = StandardScaler().fit(X2_train)
X2_train = scaler.transform(X2_train)
X2_test = scaler.transform(X2_test)

In [238]:
home_model = keras.Sequential()
# model layout for away team

home_model.add(layers.Dense(120, activation="relu"))
home_model.add(layers.Dense(80, activation="sigmoid"))
home_model.add(layers.Dense(60, activation="relu"))
home_model.add(layers.Dense(40, activation="sigmoid"))
home_model.add(layers.Dense(20, activation="relu"))
home_model.add(layers.Dense(40, activation="sigmoid"))
home_model.add(layers.Dense(60, activation="relu"))
home_model.add(layers.Dense(80, activation="sigmoid"))

home_model.add(layers.Dense(1, activation=None))

home_model.compile(
    optimizer=keras.optimizers.Adam(),
    # Loss function to minimize
    loss=keras.losses.MeanSquaredError()
)

In [239]:
HomePredictor = home_model.fit(X2_train, y2_train, epochs=50, verbose=1)

In [240]:
train_loss = HomePredictor.history["loss"][-1]
result = home_model.evaluate(X2_test, y2_test, verbose=0)

print(f"Train Loss: {train_loss:.4f}")
print(f"Test Loss: {result:.4f}")

h_predictions = model.predict(X2_test)

Train Loss: 64.6773
Test Loss: 90.9474
40/40 [==============================] - 0s 1ms/step


In [241]:
from sklearn.metrics import mean_squared_error

home_mse = mean_squared_error(y2_test,h_predictions)
home_mse

135.19076302967656

In [242]:
home_rmse = np.sqrt(home_mse)
home_rmse

11.627156274415364

In [243]:
HomePredictions = pd.DataFrame(h_predictions, index=index_test)
HomePredictions.head()

,0
summary,
buf_den2014,15.907201
sea_sf2009,25.790144
nyj_buf2007,25.330559
cle_ind2011,15.920972
car_tb2017,25.966648


In [244]:
HomePredictions = HomePredictions.rename(columns={0: 'home_points_prediction'})

In [245]:
HomePredictions.head()

,home_points_prediction
summary,
buf_den2014,15.907201
sea_sf2009,25.790144
nyj_buf2007,25.330559
cle_ind2011,15.920972
car_tb2017,25.966648


In [246]:
def get_home_score(away_team, home_team, season):
    temp_index = away_team + '_' + home_team + season
    result = HomePredictions.loc[temp_index,'home_points_prediction']
    return result

In [247]:
test_score = get_home_score('nyj', 'buf', '2007')
test_score

25.330559

In [248]:
HomePredictions.describe()

,home_points_prediction
count,1278.000000
mean,21.600067
std,4.907609
min,15.906848
25%,15.911275
50%,25.696694
75%,26.067301
max,26.078068


# Predicting Week 12

Our model has now been trained on historical data for both home points and away points. Now we can import our CSV for in-season data (up to week 11) to make predictions on future matchups (week 12). 

We will then compare the results of week 12 to our predictions.

In [283]:
wk11 = pd.read_csv('/Users/justintunley/Documents/BrainStation/Capstone/Model-ready-Wk11', index_col='summary')

In [284]:
Wk11DF = wk11.select_dtypes(include='number')
Wk11DF.head()

,PPG_Team1,Tot_TDs_PG_Team1,1st_Downs_PG_Team1,Rush_1st_Downs_PG_Team1,Pass_1st_Downs_PG_Team1,OFF_1st_by_pen_PG_Team1,3rd_Conv_Rate_Team1,4th_Conv_Rate_Team1,Pass_Comp_Rate_Team1,Pass_Yds_PG_Team1,...,Sack_Yds_Lost_PG_Team2,FG_Att_PG_Team2,FG_Good_PG_Team2,Pass_Att_PG_Team2,DEF_Pass_Att_PG_Team2,DEF_Sacks_PG_Team2,DEF_Sack_Yds_PG_Team2,DEF_FG_Att_PG_Team2,DEF_FG_Good_PG_Team2,season
summary,,,,,,,,,,,,,,,,,,,,,
ari_atl2023,17.5,1.176471,11.705882,4.176471,6.294118,1.235294,35.29,27.27,0.624642,111.176471,...,12.411765,1.294118,1.235294,18.941176,19.000000,1.235294,8.294118,1.235294,1.235294,2023
ari_bal2023,17.5,1.176471,11.705882,4.176471,6.294118,1.235294,35.29,27.27,0.624642,111.176471,...,9.000000,1.352941,1.117647,18.235294,23.647059,2.588235,20.588235,1.470588,1.411765,2023
ari_buf2023,17.5,1.176471,11.705882,4.176471,6.294118,1.235294,35.29,27.27,0.624642,111.176471,...,4.470588,1.117647,0.941176,22.470588,21.058824,2.294118,13.411765,1.117647,1.058824,2023
ari_car2023,17.5,1.176471,11.705882,4.176471,6.294118,1.235294,35.29,27.27,0.624642,111.176471,...,18.058824,1.176471,1.000000,22.235294,17.294118,1.000000,7.352941,0.882353,0.882353,2023
ari_chi2023,17.5,1.176471,11.705882,4.176471,6.294118,1.235294,35.29,27.27,0.624642,111.176471,...,11.823529,1.176471,1.117647,19.294118,23.647059,0.882353,5.235294,1.411765,1.058824,2023


In [251]:
columns_not_shared = set(X.columns) ^ set(Wk11DF.columns)
columns_not_shared
# "set()" means it worked, and all columns line up perfectly

set()

In [296]:
Wk11DF = scaler.transform(Wk11DF)

### Away Score

In [297]:
Wk12_away_predictions = model.predict(Wk11DF)
Wk12_away_predictions

31/31 [==============================] - 0s 1ms/step


array([[15.907496 ],
       [15.90747  ],
       [15.909036 ],
       [15.908466 ],
       [15.9075   ],
       [15.907607 ],
       [15.90747  ],
       [15.90747  ],
       [15.919132 ],
       [15.90747  ],
       [15.912084 ],
       [15.910781 ],
       [15.913102 ],
       [15.90747  ],
       [15.90747  ],
       [15.917703 ],
       [15.916012 ],
       [15.908739 ],
       [15.910523 ],
       [15.910514 ],
       [15.938937 ],
       [15.90747  ],
       [15.909286 ],
       [15.91793  ],
       [15.90747  ],
       [15.912709 ],
       [15.90747  ],
       [15.917624 ],
       [15.91079  ],
       [15.911404 ],
       [15.907482 ],
       [25.900259 ],
       [15.908343 ],
       [15.911653 ],
       [15.911695 ],
       [15.910989 ],
       [15.909954 ],
       [15.90747  ],
       [15.907531 ],
       [25.773508 ],
       [15.908344 ],
       [15.972781 ],
       [15.945706 ],
       [17.27318  ],
       [15.9083605],
       [15.90834  ],
       [21.685286 ],
       [17.18

In [298]:
Wk12_AwayPredict = pd.DataFrame(Wk12_away_predictions, index=wk11.index)

In [299]:
Wk12_AwayPredict = Wk12_AwayPredict.rename(columns={0:'away_prediction'})
Wk12_AwayPredict.head()

,away_prediction
summary,
ari_atl2023,15.907496
ari_bal2023,15.907470
ari_buf2023,15.909036
ari_car2023,15.908466
ari_chi2023,15.907500


In [300]:
def get_away_prediction(away_team, home_team, season):
    temp_index = away_team + '_' + home_team + season
    result = Wk12_AwayPredict.loc[temp_index,'away_prediction']
    return result

In [327]:
away_score = get_away_prediction('jax', 'hou', '2023')
# put away team first, will output away team prediction
away_score

25.203707

### Home Score

In [302]:
Wk12_home_predictions = home_model.predict(Wk11DF)
Wk12_home_predictions

31/31 [==============================] - 0s 979us/step


array([[16.657133],
       [20.433077],
       [16.49703 ],
       [16.53237 ],
       [16.692719],
       [16.530655],
       [22.22494 ],
       [24.107315],
       [16.446653],
       [20.349487],
       [16.463594],
       [16.472523],
       [16.462708],
       [19.286926],
       [19.100546],
       [16.463806],
       [16.462471],
       [16.463608],
       [16.43858 ],
       [16.453865],
       [16.452871],
       [18.857658],
       [16.437447],
       [16.453377],
       [23.740206],
       [16.445309],
       [19.074537],
       [16.45474 ],
       [16.45692 ],
       [16.464016],
       [16.664444],
       [16.480518],
       [25.981266],
       [17.169748],
       [16.613098],
       [24.215641],
       [16.769823],
       [26.563765],
       [26.007372],
       [16.474525],
       [26.729134],
       [16.480732],
       [16.536161],
       [16.485653],
       [24.880543],
       [23.719458],
       [16.482136],
       [16.473995],
       [16.483004],
       [16.582579],


In [303]:
Wk12_HomePredict = pd.DataFrame(Wk12_home_predictions, index=wk11.index)

In [304]:
Wk12_HomePredict = Wk12_HomePredict.rename(columns={0:'home_prediction'})
Wk12_HomePredict.head()

,home_prediction
summary,
ari_atl2023,16.657133
ari_bal2023,20.433077
ari_buf2023,16.497030
ari_car2023,16.532370
ari_chi2023,16.692719


# NFL Score Predictor

In [385]:
def get_home_prediction(away_team, home_team, season):
    temp_index = away_team + '_' + home_team + season
    result1 = Wk12_HomePredict.loc[temp_index,'home_prediction']
    return result1

In [386]:
def get_away_prediction(away_team, home_team, season):
    temp_index = away_team + '_' + home_team + season
    result2 = Wk12_AwayPredict.loc[temp_index,'away_prediction']
    return result2

In [445]:
away_score = get_away_prediction('sea', 'dal', '2023')
away_score
# Always put away team first

15.90747

In [444]:
home_score = get_home_prediction('sea', 'dal', '2023')
home_score
# Always put away team first

23.753992

In [446]:
expected_spread = home_score - away_score
expected_spread

7.8465223

# Prediction Summary

In [311]:
Wk12_HomePredict.describe()

,home_prediction
count,992.000000
mean,18.216728
std,3.183171
min,16.425373
25%,16.472078
50%,16.498679
75%,18.164776
max,29.341600


In [312]:
Wk12_AwayPredict.describe()

,away_prediction
count,992.000000
mean,17.823038
std,3.828997
min,15.907469
25%,15.908621
50%,15.914090
75%,16.036201
max,26.072409
